In [4]:
import json
from public_data.models import Ocsge, Commune, CouvertureSol
from django.db import connection

city = Commune.objects.get(name="Auch")
city

<Commune: Auch (32013)>

In [5]:
bbox = city.mpoly.extent
bbox

(0.486067241998179, 43.6175987830157, 0.647056360983016, 43.7076959675878)

In [17]:
query = (
    "SELECT "
    "o.id, o.couverture_label, o.usage_label, o.surface, o.year, t.map_color, "
    "st_AsGeoJSON(mpoly, 8, 0) AS geojson " 
    f"FROM {Ocsge._meta.db_table} as o INNER JOIN {CouvertureSol._meta.db_table} as t ON t.code_prefix = o.couverture "
    "WHERE mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) "
    "LIMIT 2"
)

with connection.cursor() as cursor:
    cursor.execute(query, bbox)
    rows = cursor.fetchall()
rows

[(1086245,
  "Surfaces d'eau (Eau continentale et maritime)",
  'Sans usage',
  Decimal('2712.6875'),
  2019,
  '#00ccf2',
  '{"type":"MultiPolygon","coordinates":[[[[0.50717376,43.65973629],[0.50684621,43.6595924],[0.5066509,43.65949015],[0.50661965,43.65949285],[0.50657597,43.65947367],[0.50647813,43.65939967],[0.50630884,43.65931216],[0.50626583,43.65933737],[0.50625676,43.65943187],[0.50641592,43.65969515],[0.50669721,43.66001597],[0.50685371,43.66003864],[0.50698152,43.65996532],[0.50717376,43.65973629]]]]}'),
 (1089830,
  'Formations herbacées (Pelouses et prairies; terres arables; roselières; …)',
  'Résidentiel',
  Decimal('14358.5207'),
  2019,
  '#ccf24d',
  '{"type":"MultiPolygon","coordinates":[[[[0.51216554,43.66881631],[0.51211459,43.66880713],[0.51206019,43.66879769],[0.51207277,43.66875927],[0.5121002,43.66867581],[0.51222687,43.66869771],[0.51227734,43.66870651],[0.5122702,43.66872796],[0.51225956,43.66876174],[0.51226387,43.66875968],[0.51226938,43.66875845],[0.512273

In [48]:
query = f"""
SELECT
    ST_NPoints(mpoly) AS geometry,
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.000000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.00000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.0000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.00001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.0001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.01))
FROM {Ocsge._meta.db_table} as o INNER JOIN {CouvertureSol._meta.db_table} as t ON t.code_prefix = o.couverture 
WHERE mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326)
ORDER BY ? DESC
LIMIT 10
"""
with connection.cursor() as cursor:
    cursor.execute(query, bbox)
    rows = cursor.fetchall()
rows

ProgrammingError: syntax error at or near "DESC"
LINE 14: ORDER BY ? DESC
                    ^


In [33]:
query = f"""
SELECT
    ST_NPoints(mpoly) AS geometry,
    ST_NPoints(ST_Simplify(mpoly, 0.1)),
    ST_NPoints(ST_Simplify(mpoly, 1)),
    ST_NPoints(ST_Simplify(mpoly, 10)),
    ST_NPoints(ST_Simplify(mpoly, 100))
FROM {Ocsge._meta.db_table} as o INNER JOIN {CouvertureSol._meta.db_table} as t ON t.code_prefix = o.couverture 
WHERE mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) 
LIMIT 10
"""
with connection.cursor() as cursor:
    cursor.execute(query, bbox)
    rows = cursor.fetchall()
rows

[(14, None, None, None, None),
 (154, None, None, None, None),
 (14, None, None, None, None),
 (16, None, None, None, None),
 (17, None, None, None, None),
 (14, None, None, None, None),
 (14, None, None, None, None),
 (16, None, None, None, None),
 (76, None, None, None, None),
 (18, None, None, None, None)]

In [27]:
query = """
SELECT ST_Npoints(the_geom) As np_before, ST_NPoints(ST_SimplifyPreserveTopology(the_geom,0.1)) As np01_notbadcircle, ST_NPoints(ST_SimplifyPreserveTopology(the_geom,0.5)) As np05_notquitecircle,
ST_NPoints(ST_SimplifyPreserveTopology(the_geom,1)) As np1_octagon, ST_NPoints(ST_SimplifyPreserveTopology(the_geom,10)) As np10_square,
ST_NPoints(ST_SimplifyPreserveTopology(the_geom,100)) As  np100_stillsquare
FROM (SELECT ST_Buffer('POINT(1 3)', 10,12) As the_geom) As foo;
"""
with connection.cursor() as cursor:
    cursor.execute(query, bbox)
    rows = cursor.fetchall()
rows

[(49, 33, 17, 9, 5, 5)]

In [ ]:
from rest_framework_gis.serializers import GeoFeatureModelSerializer
from ipyleaflet import Map, GeoJSON

class CustomSerializer(GeoFeatureModelSerializer):
    geom = GeometryField()
    
    class Meta:
        model = Comm
        geo_field = "geom"
        fields=('_commune', )

# get the data from the database
geom = Project.objects.get(pk=4).combined_emprise
qs = Comm.objects.filter(mpoly__intersects=geom).annotate(intersection=Intersection("mpoly", geom))
qs = qs.annotate(intersection_area=Area("intersection")).annotate(area=Area("mpoly"))
    
serializer = CustomSerializer(qs, many=True)

# Start map renderint
m = Map(center=(44.6586, -1.164), zoom=11)

geo_json = GeoJSON(
    data=serializer.data,
    style={'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1},
    hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5},
    style_callback={'color': 'black',
                    'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),}
)
m.add_layer(geo_json)
m

In [ ]:
def get_data(query, params, headers):
    with connection.cursor() as cursor:
        cursor.execute(query, params)
        return [zip(headers, row) for row in cursor.fetchall()]

def optimized(query):
        geojson = {
            "type": "FeatureCollection",
            "crs": {"type": "name", "properties": {"name": "EPSG:4326"}},
            "features": [{
                    "type": "Feature",
                    "properties": {},
                    "geometry": json.loads(row["geom"]),
                } for row in self.get_data(query)],
        }
    return geojson

In [31]:
list(zip(["a","b","c"], [1,2,3]))

[('a', 1), ('b', 2), ('c', 3)]

In [46]:
query = f"""
SELECT json_build_object(
    'type', 'FeatureCollection',
    'features', json_agg(
        json_build_object(
            'type', 'Feature',
            'geometry', ST_AsGeoJSON(mpoly)::json,
            'properties', json_build_object(
                'couverture_label', o.couverture_label,
                'usage_label', o.usage_label,
                'surface', o.surface,
                'year', o.year,
                'map_color', t.map_color
            )
        )
    )
)
FROM {Ocsge._meta.db_table} as o INNER JOIN {CouvertureSol._meta.db_table} as t ON t.code_prefix = o.couverture
LIMIT 10;
"""
with connection.cursor() as cursor:
    cursor.execute(query, bbox)
    rows = cursor.fetchall()
rows

OperationalError: out of memory
DETAIL:  Cannot enlarge string buffer containing 1073741450 bytes by 424 more bytes.
